In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import h5py
import healpy as hp
import kalepy as kale

import holodeck as holo
import holodeck.anisotropy as anis
from holodeck import detstats, plot, utils
from holodeck.constants import YR, MSOL, GYR

In [ ]:
SHAPE=20

# Setup Example Model

In [ ]:
# get frequencies
dur = holo.librarian.DEF_PTA_DUR*YR
fobs_gw_cents, fobs_gw_edges = holo.utils.pta_freqs(dur)
fobs_orb_cents = fobs_gw_cents/2
fobs_orb_edges = fobs_gw_edges/2

In [ ]:
# build Semi-Analytic-Model
sam = holo.sams.Semi_Analytic_Model(shape=SHAPE)
hard = holo.hardening.Fixed_Time_2PL_SAM(sam, 3*GYR)
if isinstance(hard, holo.hardening.Fixed_Time_2PL_SAM):
    hard_name = 'Fixed Time'
elif isinstance(hard, holo.hardening.Hard_GW):
    hard_name = 'GW Only'

# Evolve model to get final number densities & redshifts at bin edges
redz_final, dnum = holo.sam_cython.dynamic_binary_number_at_fobs(
    fobs_orb_cents, sam, hard, holo.cosmo)
redz_initl = np.array(sam.redz)[np.newaxis,np.newaxis,:,np.newaxis]
redz_prime = sam._redz_prime[...,np.newaxis]
print(f"{redz_final.shape=}, {redz_prime.shape=}, {redz_initl.shape=}")

# calculate final number per bin at bin centers
edges = [sam.mtot, sam.mrat, sam.redz, fobs_orb_edges]
number = holo.sam_cython.integrate_differential_number_3dx1d(edges, dnum)

In [ ]:
# calculate strain amplitudes

# at bin centers
hs_cents_initl = anis.strain_amp_at_bin_centers_redz(edges, None)
hs_cents_prime = anis.strain_amp_at_bin_centers_redz(edges, redz_prime)
hs_cents_final = anis.strain_amp_at_bin_centers_redz(edges, redz_final) 
print(f"{hs_cents_initl.shape=}, {hs_cents_prime.shape=}, {hs_cents_final.shape=}")

# at bin edges
hs_edges_initl = anis.strain_amp_at_bin_edges_redz(edges, None)
hs_edges_prime = anis.strain_amp_at_bin_edges_redz(edges, redz_prime)
hs_edges_final = anis.strain_amp_at_bin_edges_redz(edges, redz_final) 
print(f"{hs_edges_initl.shape=}, {hs_edges_prime.shape=}, {hs_edges_final.shape=}")

# Calculate Cl's

In [ ]:
# from number and hs at bin centers
Cl_cents_initl, ... = anis.Cl_analytic_from_num(fobs_orb_edges, number, hs_cents_initl)
Cl_cents_prime,... = anis.Cl_analytic_from_num(fobs_orb_edges, number, hs_cents_prime)
Cl_cents_final,... = anis.Cl_analytic_from_num(fobs_orb_edges, number, hs_cents_final)

In [ ]:
print(np.max(fobs_gw_cents))

In [ ]:
print(Cl_cents_final)

In [ ]:
fig = anis.plot_ClC0_versions(fobs_gw_cents, spk=True, bayes=True, sim=False, 
                              analytic=True, Cl_analytic=Cl_cents_final)
